In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = ''
!unzip "{path}"yolov7_seg.zip

Archive:  /content/drive/Shareddrives/P&I/Data & ML/PV&rooftop_project/yolov7_seg.zip
   creating: yolov7_seg/
   creating: yolov7_seg/det/
  inflating: yolov7_seg/.DS_Store    
  inflating: __MACOSX/yolov7_seg/._.DS_Store  
   creating: yolov7_seg/cls/
  inflating: yolov7_seg/README.md    
   creating: yolov7_seg/seg/
   creating: yolov7_seg/.git/
   creating: yolov7_seg/det/classify/
  inflating: yolov7_seg/det/LICENSE  
  inflating: yolov7_seg/det/requirements.txt  
  inflating: yolov7_seg/det/val.py   
   creating: yolov7_seg/det/utils/
   creating: yolov7_seg/det/models/
  inflating: yolov7_seg/det/export.py  
  inflating: yolov7_seg/det/README.md  
  inflating: yolov7_seg/det/train.py  
  inflating: yolov7_seg/det/setup.cfg  
  inflating: yolov7_seg/det/hubconf.py  
   creating: yolov7_seg/det/data/
  inflating: yolov7_seg/det/detect.py  
  inflating: yolov7_seg/cls/classifier.py  
  inflating: yolov7_seg/cls/LICENSE  
  inflating: yolov7_seg/cls/requirements.txt  
  inflating: y

In [3]:
%cd yolov7_seg/seg

/content/yolov7_seg/seg


In [ ]:
!pip install -r requirements.txt

In [ ]:
import torch

from IPython.display import Image
from IPython.display import display
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

import numpy as np
import math
import glob
import os
import random
import shutil

from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm

import warnings

random.seed(108)

##### test

In [ ]:
!cp "{path}"test_image.png /content

In [ ]:
!cp "{path}"best.onnx /content

In [ ]:
!python segment/predict.py --weights /content/best.pt --source /content/test_image.png --line-thickness 1 --save-txt --save-masks --project /content/ #--nosave-with-bbox

In [ ]:
path_inference = '/content/exp/'
masks_imgs_path = path_inference + "masks/images"
masks_txts_path = path_inference + "masks/labels"

!mkdir /content/img_check
!mkdir /content/img_check/roofs
!mkdir /content/img_check/roofs/txts
!mkdir /content/img_check/pvs
!mkdir /content/img_check/pvs/txts
!mkdir /content/img_check/pvs_neg
!mkdir /content/img_check/pvs_neg/txts

In [ ]:
list_inferred_pngs = glob.glob(f'{path_inference}*.png')
list_img_nums = [i.split("_")[0] for i in list_inferred_pngs]
print(f"there are {len(list_img_nums)} images inferred by the model.")

there are 1 images inferred by the model.


## postprocessing - get center building and check if PVs

#### only takes one image, not iterating over images!

In [ ]:
num_of_img = list_img_nums[0].split("/")[-1]
print(num_of_img)

105


In [ ]:
# check for all images in path if the name.split("_")[0] == img_num_of_interest
for img in os.listdir(masks_imgs_path):
    if img.split("_")[0] == num_of_img:
      if "Roof type" in img:
        shutil.copy(os.path.join(masks_imgs_path, img), os.path.join("/content/img_check/roofs", img))
      elif "PV_Negative" in img:
        shutil.copy(os.path.join(masks_imgs_path, img), os.path.join("/content/img_check/pvs_neg", img))
      elif "PV" in img:
        shutil.copy(os.path.join(masks_imgs_path, img), os.path.join("/content/img_check/pvs", img))
# same for .txt files ;-)
for imgtxt in os.listdir(masks_txts_path):
    if imgtxt.split("_")[0] == num_of_img:
      if "Roof type" in imgtxt:
        shutil.copy(os.path.join(masks_txts_path, imgtxt), os.path.join("/content/img_check/roofs/txts", imgtxt))
      elif "PV_Negative" in imgtxt:
        shutil.copy(os.path.join(masks_txts_path, imgtxt), os.path.join("/content/img_check/pvs_neg/txts", imgtxt))
      elif "PV" in imgtxt:
        shutil.copy(os.path.join(masks_txts_path, imgtxt), os.path.join("/content/img_check/pvs/txts", imgtxt))


In [ ]:
# get bounding box of mask
def get_bbox_points(img):  # https://stackoverflow.com/questions/31400769/bounding-box-of-numpy-array
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax

def get_bbox_array(orig_array):
  row_min, row_max, col_min, col_max = get_bbox_points(orig_array)
  img_array = np.zeros(orig_array.shape, dtype=np.uint8)
  bbox_image = np.zeros_like(img_array)
  bbox_image[row_min:row_max, col_min:col_max] = 255
  return bbox_image

def visualize_bbox(bbox_image):
  bb_pil = Image.fromarray(bbox_image)
  display(bb_pil)

def is_center_pixel_in_mask(bbox_image):
    h, w = bbox_image.shape[:2]
    center_x, center_y = w // 2, h // 2
    return bbox_image[center_y, center_x] != 0

def get_meter2pix_res(zoom, latitude):  # https://gis.stackexchange.com/questions/439792/how-to-get-metadata-google-maps-static-api-specifically-the-pixel-per-centimet
   return 156543.03 * math.cos(math.pi / 180 * latitude) / (2 ** zoom)

def is_mask_within_bbox(orig_mask, bbox_mask, threshold=.9):
    bbox_mask_binary = (bbox_mask > 0).astype(np.uint8)
    # intersection_mask = np.multiply(orig_mask, bbox_mask)
    intersection_mask = np.multiply(orig_mask, bbox_mask_binary)
    intersection_sum = np.sum(intersection_mask)
    orig_mask_sum = np.sum(orig_mask)
    if intersection_sum / orig_mask_sum >= threshold:
        return True
    else:
        return False

In [ ]:
get_meter2pix_res(20, 54.8), get_meter2pix_res(20, 47.3)

(0.08605619560540909, 0.1012431807946658)

assume 10cm per pixel accross Germany for 20zoom



In [ ]:
cm_per_pixel_at_400 = 10
cm_per_pixel = cm_per_pixel_at_400 * 400/640  # need to weight each pixel because the model resizes the images to 640!
meters_per_pixel = cm_per_pixel / 100
cm_per_pixel, meters_per_pixel


(6.25, 0.0625)

In [ ]:
count_num_roofs_in_center = 0  # should be max 1... if more print a warning ;-)
num_pixels_pv_on_center_roof = 0
for roof_mask in glob.glob(f"/content/img_check/roofs/{num_of_img}_*.png"):
  imgraw = Image.open(roof_mask)
  img_array_roof = np.array(imgraw)
  bbox_roof = get_bbox_array(img_array_roof)
  if is_center_pixel_in_mask(bbox_roof):
    count_num_roofs_in_center += 1
    list_pvs = glob.glob(f"/content/img_check/pvs/{num_of_img}_*.png")
    for pv_img in list_pvs:
      img = Image.open(pv_img)
      img_array_pv = np.array(img)
      if is_mask_within_bbox(img_array_pv,bbox_roof):
        num_pixels_pv_on_center_roof += np.count_nonzero(img_array_pv)

accumulative_area_pv_center_roof = int(num_pixels_pv_on_center_roof * meters_per_pixel * meters_per_pixel)
print("area of pv in the center roof:", accumulative_area_pv_center_roof)

if count_num_roofs_in_center > 1:
  warnings.warn(f"image number {num_of_img} has more than one roof instance in the center point, check why...")

area of pv in the center roof: 56
